In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [5]:
df = pd.read_csv('ps.csv')

In [6]:
df.head()

,Incident Number,Incident Date,Time Num,Street,Alt Street,City,County,Intersection,Mile Post,Accident Type,...,Weather Code,Pedestrian Involved,Bicycle Involved,Drug Involved,Alcohol Involved,Latitude,Longitude,Location WKT,Location,Light Condition
0,23-101216,09/15/2023 11:00:00 PM,23.0,2100 Market St,NaN,Chattanooga,Hamilton,W 21st St,NaN,Fatal,...,Clear,Yes,No,Yes,Yes,35.030156,-85.308327,POINT (-85.308327 35.030156),"(35.030156, -85.308327)",Dark - Lighted
1,23-102526,09/19/2023 04:45:00 PM,17.0,4505 10th Ave,NaN,Chattanooga,Hamilton,E 45th St,NaN,Possible Injury,...,Clear,Yes,No,No,No,34.990735,-85.288301,POINT (-85.288301 34.990735),"(34.990735, -85.288301)",Daylight
2,23-105265,09/27/2023 05:50:00 AM,6.0,4011 Rossville Blvd,NaN,Chattanooga,Hamilton,E 41st St,NaN,Suspected Minor Injury,...,Clear,Yes,No,No,No,34.995172,-85.290933,POINT (-85.290933 34.995172),"(34.995172, -85.290933)",Dark - Unknown Lighting
3,23-105569,09/27/2023 08:06:00 PM,20.0,E 3rd St,NaN,Chattanooga,Hamilton,N Holtzclaw Ave,NaN,Possible Injury,...,Rain,Yes,No,No,No,35.043663,-85.279720,POINT (-85.27972 35.043663),"(35.043663, -85.279720)",Dark - Lighted
4,23-108143,10/04/2023 04:57:00 PM,17.0,Mccallie Ave,NaN,Chattanooga,Hamilton,S Willow St,NaN,Possible Injury,...,Clear,Yes,No,No,No,35.034847,-85.271845,POINT (-85.271845 35.034847),"(35.034847, -85.271845)",Daylight


In [7]:
# get unique values for light condition
df['Light Condition'].unique()
df['Weather Code'].unique()


array(['Clear', 'Rain', 'Unknown', 'Cloudy', 'Other', 'Fog',
       'Severe Cross-Winds'], dtype=object)

In [92]:
# lets seperate indicent date into year, month, day of week, hour
df['Incident Date'] = pd.to_datetime(df['Incident Date'])
df['year'] = df['Incident Date'].dt.year
df['month'] = df['Incident Date'].dt.month
df['day'] = df['Incident Date'].dt.day
df['hour'] = df['Incident Date'].dt.hour
df['day_of_week'] = df['Incident Date'].dt.dayofweek



# replace all yes and no with 1 and 0
for col in df.columns:
    df[col] = df[col].replace('Yes', 1)
    df[col] = df[col].replace('No', 0)
    

In [93]:
# rename bicycle involved column
df.rename(columns={'Bicycle Involved':'bicycle'}, inplace=True)
df.rename(columns={'Pedestrian Involved':'pedestrian'}, inplace=True)
df.rename(columns={'Involved Fatal Injury':'fatal'}, inplace=True)
df.rename(columns={'Involved Medical Transport':'EMSTransport'}, inplace=True)
df.rename(columns={'Alcohol Involved':'etoh'}, inplace=True)
df.rename(columns={'Drug Involved':'drug'}, inplace=True)


In [97]:
# create a special df for correlation
# drop "Time Num" "Mile Post" "Accident Type" "Hit and Run" "Involved Placarded Truck" "Total Vehicles Involved"
df_corr = df.drop(['Time Num', 'Mile Post', 'Accident Type', 'Hit and Run', 'Involved Placarded Truck', 'Total Vehicles Involved','City','County','Alt Street','Incident Number',"Incident Date","Collision Type","Location WKT","Location"], axis=1)
df_corr

,Street,Intersection,fatal,EMSTransport,Posted Speed,Weather Code,pedestrian,bicycle,drug,etoh,Latitude,Longitude,Light Condition,year,month,day,hour,day_of_week
0,2100 Market St,W 21st St,1,1,35.0,Clear,1,0,1.0,1.0,35.030156,-85.308327,Dark - Lighted,2023,9,15,23,4
1,4505 10th Ave,E 45th St,0,0,30.0,Clear,1,0,0.0,0.0,34.990735,-85.288301,Daylight,2023,9,19,16,1
2,4011 Rossville Blvd,E 41st St,0,1,45.0,Clear,1,0,0.0,0.0,34.995172,-85.290933,Dark - Unknown Lighting,2023,9,27,5,2
3,E 3rd St,N Holtzclaw Ave,0,1,35.0,Rain,1,0,0.0,0.0,35.043663,-85.279720,Dark - Lighted,2023,9,27,20,2
4,Mccallie Ave,S Willow St,0,1,35.0,Clear,1,0,0.0,0.0,35.034847,-85.271845,Daylight,2023,10,4,16,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,727 E 11th St,Peeples Street,0,1,30.0,Clear,1,0,0.0,0.0,35.040477,-85.299892,Daylight,2023,8,26,13,5
630,4700 Midland Pike,N Parkdale Ave,0,1,45.0,Clear,1,0,0.0,0.0,35.027370,-85.224889,Daylight,2023,9,1,12,4
631,1700 Gunbarrel Rd,Crane Rd,0,1,35.0,Clear,1,0,0.0,0.0,35.021618,-85.160209,Daylight,2023,9,5,10,1
632,1300 Central Ave,E 13th St,1,1,55.0,Clear,1,0,0.0,0.0,35.033943,-85.295302,Dark - Unknown Lighting,2023,9,8,6,4


In [101]:
# convert light condition, weather code to numeric 
le = LabelEncoder()
df_corr['Light Condition'] = le.fit_transform(df_corr['Light Condition'])
df_corr['Weather Code'] = le.fit_transform(df_corr['Weather Code'])

#save df_corr to csv
df_corr.to_csv('df_corr.csv', index=False)
    
    



In [102]:
df.head()

,Incident Number,Incident Date,Time Num,Street,Alt Street,City,County,Intersection,Mile Post,Accident Type,...,Latitude,Longitude,Location WKT,Location,Light Condition,year,month,day,hour,day_of_week
0,23-101216,2023-09-15 23:00:00,23.0,2100 Market St,NaN,Chattanooga,Hamilton,W 21st St,NaN,Fatal,...,35.030156,-85.308327,POINT (-85.308327 35.030156),"(35.030156, -85.308327)",Dark - Lighted,2023,9,15,23,4
1,23-102526,2023-09-19 16:45:00,17.0,4505 10th Ave,NaN,Chattanooga,Hamilton,E 45th St,NaN,Possible Injury,...,34.990735,-85.288301,POINT (-85.288301 34.990735),"(34.990735, -85.288301)",Daylight,2023,9,19,16,1
2,23-105265,2023-09-27 05:50:00,6.0,4011 Rossville Blvd,NaN,Chattanooga,Hamilton,E 41st St,NaN,Suspected Minor Injury,...,34.995172,-85.290933,POINT (-85.290933 34.995172),"(34.995172, -85.290933)",Dark - Unknown Lighting,2023,9,27,5,2
3,23-105569,2023-09-27 20:06:00,20.0,E 3rd St,NaN,Chattanooga,Hamilton,N Holtzclaw Ave,NaN,Possible Injury,...,35.043663,-85.279720,POINT (-85.27972 35.043663),"(35.043663, -85.279720)",Dark - Lighted,2023,9,27,20,2
4,23-108143,2023-10-04 16:57:00,17.0,Mccallie Ave,NaN,Chattanooga,Hamilton,S Willow St,NaN,Possible Injury,...,35.034847,-85.271845,POINT (-85.271845 35.034847),"(35.034847, -85.271845)",Daylight,2023,10,4,16,2


In [69]:
df.to_csv('ps_cleaned.csv', index=False)

In [105]:
df = pd.read_csv('ps_cleaned.csv')
df.describe()



,Time Num,Mile Post,Accident Type,Collision Type,Hit and Run,fatal,EMSTransport,Involved Placarded Truck,Posted Speed,Total Vehicles Involved,...,drug,etoh,Latitude,Longitude,Light Condition,year,month,day,hour,day_of_week
count,619.000000,24.000000,634.000000,634.0,634.000000,634.000000,634.000000,634.000000,582.000000,634.000000,...,600.000000,602.000000,634.000000,634.000000,634.000000,634.000000,634.000000,634.000000,634.000000,634.000000
mean,14.470921,55.350000,2.388013,0.0,0.329653,0.063091,0.604101,0.001577,31.451890,1.029968,...,0.038333,0.117940,35.049078,-85.254170,2.577287,2020.692429,6.375394,15.058360,13.500000,2.826498
std,5.852208,82.022844,1.709812,0.0,0.470458,0.243319,0.489429,0.039715,10.617602,0.219257,...,0.192160,0.322806,0.038036,0.063136,1.873872,1.697979,3.591867,8.881246,6.219586,1.880689
min,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,15.000000,1.000000,...,0.000000,0.000000,34.985076,-85.390573,0.000000,2018.000000,1.000000,1.000000,0.000000,0.000000
25%,10.500000,1.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,25.000000,1.000000,...,0.000000,0.000000,35.023370,-85.305750,0.000000,2019.000000,3.000000,7.000000,9.000000,1.000000
50%,15.500000,1.000000,1.000000,0.0,0.000000,0.000000,1.000000,0.000000,30.000000,1.000000,...,0.000000,0.000000,35.043440,-85.271638,4.000000,2021.000000,6.000000,15.000000,14.500000,3.000000
75%,19.000000,177.050000,4.000000,0.0,1.000000,0.000000,1.000000,0.000000,40.000000,1.000000,...,0.000000,0.000000,35.060733,-85.218694,4.000000,2022.000000,10.000000,23.000000,19.000000,4.000000
max,24.000000,183.200000,5.000000,0.0,1.000000,1.000000,1.000000,1.000000,65.000000,4.000000,...,1.000000,1.000000,35.187101,-85.055304,6.000000,2023.000000,12.000000,31.000000,23.000000,6.000000


In [71]:
# lets create a sperate dataset for incidents that required medical transport and or were fatal
df_serious = df[(df['EMSTransport'] == 1) | (df['fatal'] == 1)]
df_serious.to_csv('serious.csv', index=False)
df

,Incident Number,Incident Date,Time Num,Street,Alt Street,City,County,Intersection,Mile Post,Accident Type,...,Latitude,Longitude,Location WKT,Location,Light Condition,year,month,day,hour,day_of_week
0,23-101216,2023-09-15 23:00:00,23.0,2100 Market St,NaN,Chattanooga,Hamilton,W 21st St,NaN,0,...,35.030156,-85.308327,POINT (-85.308327 35.030156),"(35.030156, -85.308327)",0,2023,9,15,23,4
1,23-102526,2023-09-19 16:45:00,17.0,4505 10th Ave,NaN,Chattanooga,Hamilton,E 45th St,NaN,1,...,34.990735,-85.288301,POINT (-85.288301 34.990735),"(34.990735, -85.288301)",4,2023,9,19,16,1
2,23-105265,2023-09-27 05:50:00,6.0,4011 Rossville Blvd,NaN,Chattanooga,Hamilton,E 41st St,NaN,4,...,34.995172,-85.290933,POINT (-85.290933 34.995172),"(34.995172, -85.290933)",2,2023,9,27,5,2
3,23-105569,2023-09-27 20:06:00,20.0,E 3rd St,NaN,Chattanooga,Hamilton,N Holtzclaw Ave,NaN,1,...,35.043663,-85.279720,POINT (-85.27972 35.043663),"(35.043663, -85.279720)",0,2023,9,27,20,2
4,23-108143,2023-10-04 16:57:00,17.0,Mccallie Ave,NaN,Chattanooga,Hamilton,S Willow St,NaN,1,...,35.034847,-85.271845,POINT (-85.271845 35.034847),"(35.034847, -85.271845)",4,2023,10,4,16,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,23-093329,2023-08-26 13:00:00,13.0,727 E 11th St,SR-700,Chattanooga,Hamilton,Peeples Street,NaN,1,...,35.040477,-85.299892,POINT (-85.299892 35.040477),"(35.040477, -85.299892)",4,2023,8,26,13,5
630,23-096050,2023-09-01 12:45:00,13.0,4700 Midland Pike,NaN,Chattanooga,Hamilton,N Parkdale Ave,NaN,1,...,35.027370,-85.224889,POINT (-85.224889 35.02737),"(35.027370, -85.224889)",4,2023,9,1,12,4
631,23-097228,2023-09-05 10:56:00,11.0,1700 Gunbarrel Rd,NaN,Chattanooga,Hamilton,Crane Rd,NaN,1,...,35.021618,-85.160209,POINT (-85.160209 35.021618),"(35.021618, -85.160209)",4,2023,9,5,10,1
632,23-098315,2023-09-08 06:25:00,6.5,1300 Central Ave,SR-1300,Chattanooga,Hamilton,E 13th St,NaN,0,...,35.033943,-85.295302,POINT (-85.295302 35.033943),"(35.033943, -85.295302)",2,2023,9,8,6,4


In [72]:
df



,Incident Number,Incident Date,Time Num,Street,Alt Street,City,County,Intersection,Mile Post,Accident Type,...,Latitude,Longitude,Location WKT,Location,Light Condition,year,month,day,hour,day_of_week
0,23-101216,2023-09-15 23:00:00,23.0,2100 Market St,NaN,Chattanooga,Hamilton,W 21st St,NaN,0,...,35.030156,-85.308327,POINT (-85.308327 35.030156),"(35.030156, -85.308327)",0,2023,9,15,23,4
1,23-102526,2023-09-19 16:45:00,17.0,4505 10th Ave,NaN,Chattanooga,Hamilton,E 45th St,NaN,1,...,34.990735,-85.288301,POINT (-85.288301 34.990735),"(34.990735, -85.288301)",4,2023,9,19,16,1
2,23-105265,2023-09-27 05:50:00,6.0,4011 Rossville Blvd,NaN,Chattanooga,Hamilton,E 41st St,NaN,4,...,34.995172,-85.290933,POINT (-85.290933 34.995172),"(34.995172, -85.290933)",2,2023,9,27,5,2
3,23-105569,2023-09-27 20:06:00,20.0,E 3rd St,NaN,Chattanooga,Hamilton,N Holtzclaw Ave,NaN,1,...,35.043663,-85.279720,POINT (-85.27972 35.043663),"(35.043663, -85.279720)",0,2023,9,27,20,2
4,23-108143,2023-10-04 16:57:00,17.0,Mccallie Ave,NaN,Chattanooga,Hamilton,S Willow St,NaN,1,...,35.034847,-85.271845,POINT (-85.271845 35.034847),"(35.034847, -85.271845)",4,2023,10,4,16,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,23-093329,2023-08-26 13:00:00,13.0,727 E 11th St,SR-700,Chattanooga,Hamilton,Peeples Street,NaN,1,...,35.040477,-85.299892,POINT (-85.299892 35.040477),"(35.040477, -85.299892)",4,2023,8,26,13,5
630,23-096050,2023-09-01 12:45:00,13.0,4700 Midland Pike,NaN,Chattanooga,Hamilton,N Parkdale Ave,NaN,1,...,35.027370,-85.224889,POINT (-85.224889 35.02737),"(35.027370, -85.224889)",4,2023,9,1,12,4
631,23-097228,2023-09-05 10:56:00,11.0,1700 Gunbarrel Rd,NaN,Chattanooga,Hamilton,Crane Rd,NaN,1,...,35.021618,-85.160209,POINT (-85.160209 35.021618),"(35.021618, -85.160209)",4,2023,9,5,10,1
632,23-098315,2023-09-08 06:25:00,6.5,1300 Central Ave,SR-1300,Chattanooga,Hamilton,E 13th St,NaN,0,...,35.033943,-85.295302,POINT (-85.295302 35.033943),"(35.033943, -85.295302)",2,2023,9,8,6,4


In [103]:
df.describe()

,Time Num,Mile Post,Hit and Run,fatal,EMSTransport,Involved Placarded Truck,Posted Speed,Total Vehicles Involved,pedestrian,bicycle,drug,etoh,Latitude,Longitude,year,month,day,hour,day_of_week
count,619.000000,24.000000,634.000000,634.000000,634.000000,634.000000,582.000000,634.000000,634.0,634.0,600.000000,602.000000,634.000000,634.000000,634.000000,634.000000,634.000000,634.000000,634.000000
mean,14.470921,55.350000,0.329653,0.063091,0.604101,0.001577,31.451890,1.029968,1.0,0.0,0.038333,0.117940,35.049078,-85.254170,2020.692429,6.375394,15.058360,13.500000,2.826498
std,5.852208,82.022844,0.470458,0.243319,0.489429,0.039715,10.617602,0.219257,0.0,0.0,0.192160,0.322806,0.038036,0.063136,1.697979,3.591867,8.881246,6.219586,1.880689
min,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,1.000000,1.0,0.0,0.000000,0.000000,34.985076,-85.390573,2018.000000,1.000000,1.000000,0.000000,0.000000
25%,10.500000,1.000000,0.000000,0.000000,0.000000,0.000000,25.000000,1.000000,1.0,0.0,0.000000,0.000000,35.023370,-85.305750,2019.000000,3.000000,7.000000,9.000000,1.000000
50%,15.500000,1.000000,0.000000,0.000000,1.000000,0.000000,30.000000,1.000000,1.0,0.0,0.000000,0.000000,35.043440,-85.271638,2021.000000,6.000000,15.000000,14.500000,3.000000
75%,19.000000,177.050000,1.000000,0.000000,1.000000,0.000000,40.000000,1.000000,1.0,0.0,0.000000,0.000000,35.060733,-85.218694,2022.000000,10.000000,23.000000,19.000000,4.000000
max,24.000000,183.200000,1.000000,1.000000,1.000000,1.000000,65.000000,4.000000,1.0,0.0,1.000000,1.000000,35.187101,-85.055304,2023.000000,12.000000,31.000000,23.000000,6.000000


In [108]:
# COUNT INCIDENTS WHERE ALCOHOL OR DRUGS WERE INVOLVED
df['etoh'].value_counts()





0.0    531
1.0     71
Name: etoh, dtype: int64

In [109]:
df['drug'].value_counts()


0.0    577
1.0     23
Name: drug, dtype: int64